<a href="https://colab.research.google.com/github/simonh10/aivideotoys/blob/main/SceneDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Scene detection
Using a simple Discriminator network to look at the five frames before a potential cut and the five frames after a potential cut to workout if there is a scene change there.

# Prepare Data
Find a video to convert to frames and split them into scenes by hand. We can work at a 64x64 greyscale image resolution.